In [ ]:
# Emma Abraham (ea7ga) and Andrew Chaphiv (ac2gq)
#Accessing the Kaggle dataset without having to download the dataset outside of the script
#Method implementation source: https://medium.com/analytics-vidhya/fetch-data-from-kaggle-with-python-9154a4c610e3
#https://towardsdatascience.com/downloading-datasets-from-kaggle-for-your-ml-project-b9120d405ea4
#Dataset: https://www.kaggle.com/datasets/thedevastator/the-ultimate-netflix-tv-shows-and-movies-dataset
#Install kaggle to access the Kaggle Api and the package dfply if you do not already have them downloaded
#! pip install kaggle
# ! pip3 install dfply
#! pip3 install pymongo

In [1]:
#Import necessary packages/libraries
import pandas as pd
import numpy as np
import pymongo
import pprint
from zipfile import ZipFile

In [ ]:
#Used an API token to connect to kaggle to download the dataset
from kaggle.api.kaggle_api_extended import KaggleApi
#Follow the directions on this website: https://www.kaggle.com/docs/api and this website: https://medium.com/analytics-vidhya/fetch-data-from-kaggle-with-python-9154a4c610e3
#To create kaggle authentication tokens to access kaggle easily. 
api = KaggleApi()
api.authenticate()
#Download the kaggle dataset files by using the kaggle api
#to find the name and file name for the kaggle command, select copy the api command from the drop down
api.dataset_download_files('thedevastator/the-ultimate-netflix-tv-shows-and-movies-dataset')

In [ ]:
#Extract the files from the zip
zipFile = ZipFile('the-ultimate-netflix-tv-shows-and-movies-dataset.zip')
zipFile.extract('Best Movies Netflix.csv') #Extract only the files we need from zip file
zipFile.extract('Best Shows Netflix.csv')
zipFile.close()
#Now all the files are available in the same directory as this script in .csv format 

In [ ]:
#Alternative method for getting the CSV files: 
#download the files: 'Best Movies Netflix.csv' and 'Best Shows Netflix.csv' to the same folder as this script 
#And comment out the two previous chunks

In [3]:
#Part 1: Extract and Transform the data
#Read in our datasets 
movies = pd.read_csv('Best Movies Netflix.csv')
shows = pd.read_csv('Best Shows Netflix.csv')
#drop unused columns
movies.drop(['index','NUMBER_OF_VOTES','MAIN_PRODUCTION'],axis=1, inplace= True)
movies["Type"] = "Movie"
shows.drop(['index','NUMBER_OF_VOTES','NUMBER_OF_SEASONS','MAIN_PRODUCTION'],axis=1, inplace= True)
shows["Type"] = "Show"
#combine movie and show data 
new_dataset = pd.concat([movies, shows],axis=0)

In [4]:
#Part 2: Load the Data into MongoDB

#Connect to Mongodb and create local instance of db
#Reference: https://github.com/jasonwnc/ds2002-1/blob/main/nosql/06-Python-MongoDB.ipynb
host_name = "localhost"
port = "27017"

atlas_cluster_name = "sandbox"
atlas_default_dbname = "local"
#atlas_user_name = "m001-student"
#atlas_password = "m001-mongodb-basics"
conn_str = {
    "local" : f"mongodb://{host_name}:{port}/",
#    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.zibbf.mongodb.net/{atlas_default_dbname}"
}

client = pymongo.MongoClient(conn_str["local"])

print(f"Local Connection String: {conn_str['local']}")
#print(f"Atlas Connection String: {conn_str['atlas']}")

Local Connection String: mongodb://localhost:27017/


In [58]:
db_name = "chatBoxResponses"

db = client[db_name]
m_collection = db["Media"]

In [67]:
result = m_collection.find({'RELEASE_YEAR':2022-10,"Type": "Movie"})
result[0]

{'_id': ObjectId('639aa9c65df88bf2596da85d'),
 'TITLE': 'Django Unchained',
 'RELEASE_YEAR': 2012,
 'SCORE': 8.4,
 'DURATION': 165,
 'MAIN_GENRE': 'western',
 'Type': 'Movie'}

In [8]:
new_dictionary = new_dataset.to_dict('records')

In [9]:
x = m_collection.insert_many(new_dictionary)

In [ ]:
#for index in new_dictionary.index:
#    media = {"Title": new_dictionary["TITLE"][index],
#        "Release Year": new_dictionary["RELEASE_YEAR"][index],
#        "Score": new_dictionary["SCORE"][index],
#        "Duration": new_dictionary["DURATI E"][index],
#        "Media Type": new_dictionary["Type"][index]
#       }
#    x = m_collection.insert_one(media)